In [93]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [94]:
import random, copy
from Mylib import *
# %matplotlib inline

#### 구글 API: 한번만 돌림

In [95]:
GSAPI = GgleAPI('credentials.json', 'sheets')
GSAPI.Create_Service('v4', ['https://www.googleapis.com/auth/spreadsheets'])

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=780794052173-vq88qrrbro0ukaan0jltei7g2vpmrhik.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets&state=Sn5d0iyM6XA76tpOPmKsCIVjKCDSXl&access_type=offline
sheets service created successfully


#### 데이터 준비

Data Preparation

In [143]:
raw_data = pd.read_pickle('./result/HWC_noun_token0422.pkl')
raw_data = raw_data.sort_values(['CONFER_NUM', 'SEQ_NO'])
print(f'There are {sum([len(doc) for doc in raw_data.token])} tokens before processing')

manual_rms = pd.read_csv('./data/190915_용어_수작업.csv') # Manual term removals
manual_rms = manual_rms[manual_rms['code'] == 1]['단어'].values 
manual_rms = [rm for rm in manual_rms if len(rm) == 1]

# Token cleansing

speakers = set([s for tkns in [spker.split(' ') for spker in raw_data.SPEAKER] for s in tkns if len(s) <= 3 and s not in ['위원장', '위원', '의원', '진술인', '참고인']])
manual_rms = np.append(manual_rms, 
                       ["계속", '료', '박정', '말씀', '생각', '얘기', '질의', '발언', '국민']
                       + list(speakers)
                       + ['泰', '金', '강길부', '강병원', '강석호', '강창일', '강훈식', '경대수', '고용진', '곽대훈', '곽상도', '권석창', '권성동', '권은희', '권칠승', '권칠승',
                          '금태섭', '김경진', '김경협', '김관영', '김규환', '김기선', '김도읍', '김두관', '김무성', '김민기', '김병관', '김병기', '김병욱', '김부겸', '김삼화', 
                          '김석기', '김선동', '김성수', '김성식', '김성원', '김성원', '김성찬', '김성태', '김수민', '김영우', '김영진', '김영호', '김용태', '김재경', '김재원', 
                          '김정우', '김정재', '김종대', '김종민', '김종석', '김종회', '김종훈', '김중', '김진태', '김진표', '김철민', '김태년', '김태흠', '김학용', '김한표', 
                          '김해영', '김현미', '김현아', '나경원', '노웅래', '노웅래', '노회찬', '도자', '문미옥', '문진국', '문희상', '민경욱', '민병두', '민홍철', '박경미', 
                          '박광온', '박남춘', '박대출', '박덕흠', '박맹우', '박명재', '박선숙', '박성중', '박순자', '박영선', '박완수', '박완주', '박용진', '박재호', '박주민', 
                          '박주선', '박주현', '박준영', '박지원', '박찬대', '박찬우', '박홍근', '백혜련', '변재일', '보험법', '서영교', '서청원', '설훈', '소병훈', '소병훈',
                          '손금주', '손혜원', '송갑석', '송기헌', '송영길', '송옥주', '송희경', '신경민', '신보라', '신창현', '심기준', '심상정', '심재', '심재철', '안규백', 
                          '안민석', '안상수', '안호영', '어기구', '엄용수', '여상규', '오세정', '오신환', '오영훈', '우원식', '원유철', '원혜영', '위성곤', '유기준', '유동수', 
                          '유민봉', '유성엽', '유승민', '유승희', '유은혜', '윤관석', '윤상직', '윤상현', '윤영석', '윤영일', '윤재옥', '윤종오', '윤호중', '윤후덕', '이개호', 
                          '이군현', '이규희', '이동섭', '이만희', '이석현', '이수혁', '이양수', '이언주', '이완영', '이용득', '이용주', '이용호', '이원욱', '이은권', '이인영',
                          '이재정', '이정미', '이종걸', '이종구', '이종명', '이종배', '이진복', '이찬열', '이채익', '이철규', '이철희', '이춘석', '이태규', '이학영', '이학재', 
                          '이해찬', '이헌승', '이현재', '이혜훈', '이훈', '임이자', '임종성', '장병완', '장석춘', '장제원', '전재수', '전해철', '전현희', '전희경', '정갑윤', 
                          '정동영', '정병국', '정성호', '정애', '정양석', '정우택', '정운천', '정유섭', '정인화', '정재호', '정진석', '정태옥', '제윤경', '조경태', '조배숙', 
                          '조승래', '조정식', '조훈현', '주광덕', '주승용', '주승용', '주호영', '지상욱', '진선미', '체온계', '최경환', '최명길', '최연혜', '최운열', '최인호', 
                          '최재성', '추경호', '추미애', '표창원', '하태경', '한선교', '한정애', '한홍', '함진규', '혜선', '홍문종', '홍문표', '홍의락', '홍익표', '홍철호', 
                          '황영철', '황주홍', '황주홍', '황희'])

raw_data.token = [[tkn for tkn in doc if tkn not in manual_rms] for doc in tqdm(raw_data.token)]
raw_data = raw_data.loc[[idx for idx in raw_data.index if raw_data.PARTY.loc[idx] != '非의원']]
raw_data.token = [[tkn for tkn in doc if tkn != []] for doc in raw_data.token]
print(f'There are {sum([len(doc) for doc in raw_data.token])} tokens after processing')

  4%|▍         | 608/15945 [00:00<00:06, 2460.58it/s]

There are 370017 tokens before processing


100%|██████████| 15945/15945 [00:04<00:00, 3297.43it/s]


There are 184655 tokens after processing


#### 문서의 길이
말뭉치 결합 $\rightarrow$ 명사 토큰 N = 100 기준으로 재결합 $\rightarrow$ D > 2000? $\rightarrow$ Fit!\
"... it is theoretically $impossible$ to guarantee identification of topics from a small number of documents, no matter how long."\
"... poor performance of the LDA is expected when documents are too short, ..."
* 참고문헌: Jian Tang et al. (2014) Understanding the limiting factors of topic modeling via posterior contraction analysis. $\textit{In Proceedings of the 31st International Conference on International Conference on Machine Learning}$ - Volume 32 (ICML’14). JMLR.org, I–190–I–198.

In [131]:
def nrmlzcorp(corpus, thrsh = 100):
    merged_tokens = my_list(corpus).sum()
    nrmcorp, i, stop = [[]], 0, False
    while not stop:
        while len(nrmcorp[-1]) < thrsh and i < len(merged_tokens):
            nrmcorp[-1].append(merged_tokens[i])
            i += 1
            if i == len(merged_tokens):
                stop = True
        nrmcorp.append([])
    return(nrmcorp[:-1])

#### 준비 및 튜닝

In [20]:
project = '20대_보건복지부_발언정제'

# random_seed = random.randint(1, 10**4)
random_seed = 8303

# tpc_grd = [int(i) for i in np.linspace(start = 40, stop = 9, num = 32)] # Remark: decreasing order!
tpc_grd = [16]

save = True

iterators = [(ng,alp) 
             for ng in [1, 2][:1]
             for alp in ['symmetric', 'asymmetric'][:1]]

nrmcorp = nrmlzcorp(raw_data.token)
data = raw_data.copy()

for i in tqdm(iterators, 'iterations'):
    ngram, alpha = i
#     1.2 ngrams model
    corp_forLDA = make_ngram(nrmcorp, ngram, mincnt = 1, thrsh = 2)

#     2. Fiting
#     Testing?
#     corp_forLDA = corp_forLDA[0::int(len(corp_forLDA)/20)]

#     2.1. Prepare doLDA instance and hyper parameters
    inst = doLDA(corp_forLDA, use_tfidf=False, top_n_words=20)
    inst.get_bow(no_below = 0, no_above = 1)

#     2.2. Tune, get topic DF and optimal solutions
    flda, topicDF, optparset = LDA_tuning(inst_doLDA=inst, 
                                          weight_on_coher=1.,
                                          ntopics= tpc_grd,
                                          hypars = {'alpha':alpha, 'beta':['auto']},
                                          rand = random_seed,
                                          iters = 2*10**2)
    plt.close()

#     3. Save files
if save:
    for i, (전후, 회차) in enumerate(
        [('정권교체전', [343, 351]), ('정권교체후', [352, 356]), ('전체', [343, 356])]
    ):
        gdata = data.loc[[idx
                          for n in range(회차[0],회차[1]+1)
                          for idx, m in zip(data.MEETING_NAME.index, data.MEETING_NAME)
                          if re.search(str(n), m) is not None]]
        doC =  {'{}'.format(p):nrmlzcorp(gdata.loc[[idx
                                                    for idx in gdata.index
                                                    if gdata.loc[idx].PARTY == p]].token, thrsh = 50)
                for p in np.unique(gdata.PARTY)}
        if i == 0:
            tops = fitdocs(inst, flda, doC, True).assign(ba = 전후)
            tops[[tops.columns[-1]] + tops.columns[:-1].to_list()]
        else:
            tops = tops.append(fitdocs(inst, flda, doC, True).assign(ba = 전후))
            tops[[tops.columns[-1]] + tops.columns[:-1].to_list()]
            
results = {'project':project, 'flda':flda, 'inst':inst, 'optparset':optparset, 'ngram':ngram, 'random_seed':random_seed, 
           'mkdir':True, 'write':True, 'ldavis':True}
data.to_pickle(f'./result/data_{project}.pkl')
tmstmp = save_results(**results)
GSAPI.MakeSheet({'topicDF':topicDF, 'topics_top5':tops.fillna('')}, 
                sheet_title = tmstmp + '_' + project, RANGE_NAME = 'A1:AA1000')
print(', '.join([o for o, v in results.items() if v != False]) + ' saved!')

---
### 모형 활용1. 정당별

In [144]:
#     3. Save files
data = raw_data.copy()
for i, (전후, 회차) in enumerate(
    [('정권교체전', [343, 351]), ('정권교체후', [352, 356]), ('전체', [343, 356])]
):
    gdata = data.loc[[idx
                      for n in range(회차[0],회차[1]+1)
                      for idx, m in zip(data.MEETING_NAME.index, data.MEETING_NAME)
                      if re.search(str(n), m) is not None]]
    doC =  {'{}'.format(p):nrmlzcorp(gdata.loc[[idx
                                                for idx in gdata.index
                                                if gdata.loc[idx].PARTY == p]].token, thrsh = 50)
            for p in np.unique(gdata.PARTY)}
    if i == 0:
        tops = fitdocs(inst, flda, doC, True).assign(ba = 전후)
        tops[[tops.columns[-1]] + tops.columns[:-1].to_list()]
    else:
        tops = tops.append(fitdocs(inst, flda, doC, True).assign(ba = 전후))
        tops[[tops.columns[-1]] + tops.columns[:-1].to_list()]

GSAPI.MakeSheet({'topics_top5':tops.fillna('')}, 
                sheet_title = '0516_' + project, RANGE_NAME = 'A1:AA1000')

Sheet "0516_20대_보건복지부_발언정제" successfully Created


---
### 모형 활용2. 의원 상세정보 활용 - 성별, 지역구, 보건의료종사경력

In [101]:
raw_data = pd.read_pickle('./result/HWC_noun_token0422.pkl')
with open('./result/200427_0005_20대_보건복지부_발언정제/inst.pickle', 'rb') as file:
    inst = pickle.load(file)
flda = gs.models.LdaModel.load('./result/200427_0005_20대_보건복지부_발언정제/ldafit.pickle')

In [47]:
지역구정보 = pd.read_excel('./data/MPs.xls', index_col=None, header=0)
지역구정보 = 지역구정보.drop('구성', 1)
지역구정보 = 지역구정보.set_axis(['SPEAKER', '소속정당(선수, 지역구)', '성별', '보건의료계종사경력'], axis = 'columns')

지역구별view = copy.deepcopy(raw_data.loc[raw_data.PARTY != '非의원'])
# 지역구별view = 지역구별view.assign(indices = 지역구별view.index)
지역구별view.SPEAKER = [s 
                    for tkns in [spker.split(' ') for spker in 지역구별view.SPEAKER]
                    for s in tkns if len(s) <= 3 and s not in ['위원장', '위원', '의원', '진술인', '참고인']]

지역구별view = 지역구별view.merge(지역구정보)

In [19]:
보건유무, 지역구여부 = [], []
for 지역구, 보건 in 지역구별view[['소속정당(선수, 지역구)', '보건의료계종사경력']].values:
    if 지역구.find('비례') > -1:
        지역구여부.append('비례대표')
    else:
        지역구여부.append('지역구')
    if 보건 != '없음':
        보건유무.append('있음')
    else:
        보건유무.append('없음')
지역구별view = 지역구별view.assign(지역구여부 = 지역구여부)
지역구별view = 지역구별view.assign(보건의료계종사경력유무 = 보건유무)
f'{len(지역구별view)} documents found in 지역구별view'

'9621 documents found in 지역구별view'

In [20]:
data = 지역구별view
Tops = pd.DataFrame()
for column, project in tqdm(zip(['지역구여부', '성별', '보건의료계종사경력유무'], [f'20대_보건복지부_{p}' for p in ['지역구', '성별', '보건의료경력']])):
    for i, (전후, 회차) in enumerate(
        [('20대_보건복지부', [343, 356])]
    ):
        gdata = data.loc[[idx
                          for n in range(회차[0],회차[1]+1)
                          for idx, m in zip(data.MEETING_NAME.index, data.MEETING_NAME)
                          if re.search(str(n), m) is not None]]
        doC =  {f'{g1}':nrmlzcorp(gdata.loc[[idx
                                             for idx in gdata.index
                                             if gdata.loc[idx][column] == g1]].token, thrsh = 50)
                for g1 in np.unique(gdata[column])}
        if i == 0:
            tops = fitdocs(inst, flda, doC, True).assign(ba = 전후)
            tops[[tops.columns[-1]] + tops.columns[:-1].to_list()]
        else:
            tops = tops.append(fitdocs(inst, flda, doC, True).assign(ba = 전후))
            tops[[tops.columns[-1]] + tops.columns[:-1].to_list()]
        tops = tops[[tops.columns[-1]] + tops.columns[:-1].to_list()]
    Tops = Tops.append(tops)

3it [00:22,  7.32s/it]


In [22]:
Tops.to_html('./result/추가분석.html', encoding='utf-16')

print([re.findall("[^+*0-9.\" ]+", j) for i, j in flda.print_topics(num_words = 20)])